In [2]:
from broadcaster_nets_2_3 import *
from BCMPLayer import *

from torch_geometric.datasets import Planetoid

import numpy as np

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

import torch.nn as N

import itertools

import csv
from time import perf_counter


from layer2Test import netTest2layer
from netsGenerator import *
from time import perf_counter
import sys
import copy

import faiss
import faiss.contrib.torch_utils


import time
import torch

# import os
# os.environ['CUDA_PATH'] = '/modules/apps/cuda/11.3.1' 
# import pykeops
# from pykeops.torch import LazyTensor
# # pykeops.clean_pykeops() 
# pykeops.test_torch_bindings()

from guppy import hpy
from matplotlib import pyplot as plt

print(torch.cuda.is_available())
!nvidia-smi


True
Mon May  8 19:09:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1A:00.0 Off |                  N/A |
| 25%   38C    P5    12W / 250W |      4MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [7]:
# import gc
# for obj in gc.get_objects():
#     try:
#         if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#             print(type(obj), obj.size())
#     except:
#         pass
# print(torch.cuda.memory_summary(device=None, abbreviated=False))

<class 'torch.Tensor'> torch.Size([2708, 1433])
<class 'torch.Tensor'> torch.Size([2, 10556])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708, 1433])
<class 'torch.Tensor'> torch.Size([2, 10556])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([2708])
<class 'torch.Tensor'> torch.Size([19])
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Us

In [32]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cora = dataset[0].to(device)

In [92]:
a = torch.arange(100,dtype=float).reshape(10,10)
print(a)
b = torch.arange(100,dtype=float).reshape(10,10)
# print(torch.stack((a,b),dim=0))
print(torch.mean(torch.stack((a,b),dim=2),2))


tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
        [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
        [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
        [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
        [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
        [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
        [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]],
       dtype=torch.float64)
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
        [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
        [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
        [60

In [15]:
!python testing2layer.py br2_$SLURM_JOB_ID Cora 1 13 GCN -1 0 .01 .001 

br2_7409325; Cora; iter:1; net set:13; conv type:GCN, gpu:NVIDIA GeForce GTX 1080 Ti, cluster:-1, cluster wait:0, lr:0.01, lr decay:0.001 
loop:0 Accuracy: 0.7800 Time: 2.6141
average: 0.78,...0,... [[0], [0]]...2.614095650613308, ... 174.0 ... 0.803 ... 0.78
----------.....................e.......................--------
loop:0 Accuracy: 0.7850 Time: 11.6329
average: 0.785,...0,... [[0], [400]]...11.63291851291433, ... 158.0 ... 0.8 ... 0.785
----------.....................e.......................--------
loop:0 Accuracy: 0.7810 Time: 32.9483
average: 0.781,...0,... [[0], [100]]...32.94832214200869, ... 90.0 ... 0.805 ... 0.781
----------.....................e.......................--------
loop:0 Accuracy: 0.7440 Time: 17.7456
average: 0.744,...0,... [[0], [400, 400]]...17.745575070381165, ... 18.0 ... 0.785 ... 0.744
----------.....................e.......................--------
loop:0 Accuracy: 0.7780 Time: 41.1564
average: 0.778,...0,... [[0], [400, 100]]...41.15643977513537, ...

In [13]:
!python testing2layer.py br2_$SLURM_JOB_ID Cora 1 12 GCN -1 0 .01 .001 

br2_7408735; Cora; iter:1; net set:12; conv type:GCN, gpu:NVIDIA GeForce GTX 1080 Ti, cluster:-1, cluster wait:0, lr:0.01, lr decay:0.001 
loop:0 Accuracy: 0.7870 Time: 1.7233
average: 0.787,...0,... [[0], [0]]...1.7232611989602447, ... 113.0 ... 0.805 ... 0.787
----------.....................e.......................--------
loop:0 Accuracy: 0.7930 Time: 9.4402
average: 0.793,...0,... [[0], [400]]...9.440233016386628, ... 90.0 ... 0.821 ... 0.793
----------.....................e.......................--------
loop:0 Accuracy: 0.7860 Time: 14.3207
average: 0.786,...0,... [[0], [200]]...14.32068578992039, ... 182.0 ... 0.809 ... 0.786
----------.....................e.......................--------
loop:0 Accuracy: 0.7510 Time: 23.6545
average: 0.751,...0,... [[0], [100]]...23.654540649149567, ... 115.0 ... 0.784 ... 0.751
----------.....................e.......................--------
loop:0 Accuracy: 0.7860 Time: 1.4971
average: 0.786,...0,... [[400], [0]]...1.4971223659813404, ... 106.